In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/PAYTM_historical_data.csv")
data

,Date,Open,High,Low,Close,Volume
0,2022-08-01T09:15:00+05:30,708.00,713.90,707.45,712.75,47041
1,2022-08-01T09:16:00+05:30,713.40,714.00,710.30,711.90,19707
2,2022-08-01T09:17:00+05:30,711.05,712.10,711.00,711.30,14889
3,2022-08-01T09:18:00+05:30,712.00,712.65,711.20,711.95,11883
4,2022-08-01T09:19:00+05:30,712.35,713.55,712.15,712.55,14532
...,...,...,...,...,...,...
184495,2023-07-31T15:25:00+05:30,644.80,644.80,644.15,644.15,19116
184496,2023-07-31T15:26:00+05:30,644.35,644.70,644.15,644.70,22111
184497,2023-07-31T15:27:00+05:30,644.70,644.70,644.35,644.40,24553
184498,2023-07-31T15:28:00+05:30,644.40,644.65,642.90,644.00,67930


In [9]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load historical data from CSV
data = pd.read_csv('/content/drive/MyDrive/PAYTM_historical_data.csv')
data['Date'] = pd.to_datetime(data['Date'])

data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')
data.fillna(0, inplace=True)

# Calculate actual returns based on Close prices
data['Actual_Returns'] = data['Close'].pct_change().fillna(0)

# Define the template function for evaluating strategies
def evaluate_strategy(strategy_name, strategy_func, data, actual_returns):
    # Implement the strategy logic
    predicted_returns = strategy_func(data)
    # Define evaluation metrics function

    accuracy = accuracy_score(actual_returns > 0, predicted_returns > 0)
    precision = precision_score(actual_returns > 0, predicted_returns > 0)
    recall = recall_score(actual_returns > 0, predicted_returns > 0)
    f1 = f1_score(actual_returns > 0, predicted_returns > 0)

    positive_trades = ((actual_returns > 0) & (predicted_returns > 0)).sum()
    total_trades = len(data)
    profit_percentage = (positive_trades / total_trades) * 100

# Print evaluation results for the strategy
    print(f"Strategy: {strategy_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Profit Percentage: {profit_percentage:.2f}%\n")

# Define the function for the Basic Breakout Trading Strategy
def basic_breakout_strategy(data, lookback_period=10, breakout_multiplier=0.9):
    signals = pd.DataFrame(index=data.index)
    signals['RollingHigh'] = data['High'].rolling(window=lookback_period).max()
    signals['RollingLow'] = data['Low'].rolling(window=lookback_period).min()
    signals['BreakoutLevel'] = signals['RollingHigh'] + breakout_multiplier * (signals['RollingHigh'] - signals['RollingLow'])

    signals['BuySignal'] = data['Close'] > signals['BreakoutLevel']
    signals['SellSignal'] = data['Close'] < signals['BreakoutLevel']

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Trend Following Strategy
def trend_following_strategy(data):
    signals = pd.DataFrame(index=data.index)
    signals['Short_MA'] = data['Close'].rolling(window=50).mean()
    signals['Long_MA'] = data['Close'].rolling(window=200).mean()

    signals['BuySignal'] = signals['Short_MA'] > signals['Long_MA']
    signals['SellSignal'] = signals['Short_MA'] < signals['Long_MA']

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Momentum Trading Strategy
def momentum_trading_strategy(data, lookback_period=20, threshold=4):
    signals = pd.DataFrame(index=data.index)
    signals['PriceChange'] = data['Close'].diff(lookback_period)
    signals['BuySignal'] = signals['PriceChange'] > threshold
    signals['SellSignal'] = signals['PriceChange'] < -threshold

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Pivot Point Trading Strategy
def pivot_point_trading_strategy(data):
    signals = pd.DataFrame(index=data.index)
    signals['Pivot'] = (data['High'] + data['Low'] + data['Close']) / 3
    signals['Previous_Close'] = data['Close'].shift(1)

    signals['BuySignal'] = (data['High'] > signals['Pivot']) & (signals['Pivot'] > signals['Previous_Close'])
    signals['SellSignal'] = (data['Low'] < signals['Pivot']) & (signals['Pivot'] < signals['Previous_Close'])

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Range Breakout Strategy
def range_breakout_strategy(data, lookback_period=12, multiplier=5):
    signals = pd.DataFrame(index=data.index)
    signals['High'] = data['High'].rolling(window=lookback_period).max()
    signals['Low'] = data['Low'].rolling(window=lookback_period).min()
    signals['Range'] = signals['High'] - signals['Low']
    signals['BreakoutLevel'] = signals['High'] + multiplier * signals['Range']

    signals['BuySignal'] = data['Close'] > signals['BreakoutLevel']
    signals['SellSignal'] = data['Close'] < signals['BreakoutLevel']

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Options Straddle Strategy
def options_straddle_strategy(data, strike_price=95, days_to_expiry=60, threshold=10):
    signals = pd.DataFrame(index=data.index)
    # Define parameters for the options straddle strategy
    strike_price = data['Close'].iloc[-1]  # Use the most recent closing price as the strike price

    # Calculate potential profit and loss based on the strategy
    signals['Call_Price'] = data['Close'] - strike_price
    signals['Put_Price'] = strike_price - data['Close']

    # Calculate the straddle position profit based on the difference between call and put prices
    signals['Straddle_Position'] = signals['Call_Price'] + signals['Put_Price']

    # Generate buy signal if the straddle position profit exceeds the threshold
    signals['BuySignal'] = signals['Straddle_Position'] > threshold
    signals['SellSignal'] = signals['Straddle_Position'] < -threshold

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Gap Trading Strategy
def gap_trading_strategy(data):
    signals = pd.DataFrame(index=data.index)
    signals['Open_Previous'] = data['Open'].shift(1)
    signals['Gap_Up'] = data['Open'] > signals['Open_Previous']
    signals['Gap_Down'] = data['Open'] < signals['Open_Previous']

    # Calculate predicted returns based on signals
    predicted_returns = signals['Gap_Up'].astype(int) - signals['Gap_Down'].astype(int)

    return predicted_returns

# Define the function for the Seasonal Trading Strategy
def seasonal_trading_strategy(data, window_size=10):
    signals = pd.DataFrame(index=data.index)
    signals['Returns'] = data['Actual_Returns'].rolling(window=window_size).sum()

    # Generate buy signal if recent returns exceed historical average
    signals['BuySignal'] = signals['Returns'] > signals['Returns'].mean()

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int)

    return predicted_returns

# Define the function for the Volatility Breakout Strategy
def volatility_breakout_strategy(data, lookback_period=16, multiplier=6):
    signals = pd.DataFrame(index=data.index)
    signals['High'] = data['High'].rolling(window=lookback_period).max()
    signals['Low'] = data['Low'].rolling(window=lookback_period).min()
    signals['Volatility'] = signals['High'] - signals['Low']

    signals['BuySignal'] = data['Close'] > signals['High'] + multiplier * signals['Volatility']
    signals['SellSignal'] = data['Close'] < signals['Low'] - multiplier * signals['Volatility']

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'].astype(int) - signals['SellSignal'].astype(int)

    return predicted_returns

# Define the function for the Trend Reversal Strategy
def trend_reversal_strategy(data):
    signals = pd.DataFrame(index=data.index)

    # Calculate the rate of price change
    data['PriceChange'] = data['Close'].diff()

    # Define a threshold for identifying potential trend reversals
    threshold = 0.5

    # Generate buy signal if there's a significant negative price change
    signals['BuySignal'] = np.where(data['PriceChange'] < -threshold, 1, 0)
    signals['SellSignal'] = np.where(data['PriceChange'] > threshold, 1, 0)

    # Calculate predicted returns based on signals
    predicted_returns = signals['BuySignal'] - signals['SellSignal']

    return predicted_returns


# Call the evaluate_strategy function for each strategy
evaluate_strategy("Basic Breakout Trading", basic_breakout_strategy, data, data['Actual_Returns'])
evaluate_strategy("Trend Following Strategy", trend_following_strategy, data, data['Actual_Returns'])
evaluate_strategy("Momentum Trading Strategy", momentum_trading_strategy, data, data['Actual_Returns'])
evaluate_strategy("Pivot Point Trading Strategy", pivot_point_trading_strategy, data, data['Actual_Returns'])
evaluate_strategy("Range Breakout Strategy", range_breakout_strategy, data, data['Actual_Returns'])
evaluate_strategy("Options Straddle Strategy", options_straddle_strategy, data, data['Actual_Returns'])
evaluate_strategy("Gap Trading Strategy", gap_trading_strategy, data, data['Actual_Returns'])
evaluate_strategy("Seasonal Trading Strategy", seasonal_trading_strategy, data, data['Actual_Returns'])
evaluate_strategy("Volatility Breakout Strategy", volatility_breakout_strategy, data, data['Actual_Returns'])
evaluate_strategy("Trend Reversal Strategy", trend_reversal_strategy, data, data['Actual_Returns'])

strategy_results=[
    {
        "Strategy": "Basic Breakout Trading",
        "Accuracy": 0.5554,
        "Precision": 0.0000,
        "Recall": 0.0000,
        "F1 Score": 0.0000,
        "Profit Percentage": 0.00,
    },
    {
        "Strategy": "Trend Following Strategy",
        "Accuracy": 0.4995,
        "Precision": 0.4416,
        "Recall": 0.4747,
        "F1 Score": 0.4575,
        "Profit Percentage": 21.11,
    },
    {
        "Strategy": "Momentum Trading Strategy",
        "Accuracy": 0.5606,
        "Precision": 0.5577,
        "Recall": 0.0564,
        "F1 Score": 0.1024,
        "Profit Percentage": 2.51,
    },
    {
        "Strategy": "Pivot Point Trading Strategy",
        "Accuracy": 0.8956,
        "Precision": 0.8601,
        "Recall": 0.9138,
        "F1 Score": 0.8862,
        "Profit Percentage": 40.63,
    },
    {
        "Strategy": "Range Breakout Strategy",
        "Accuracy": 0.5554,
        "Precision": 0.0000,
        "Recall": 0.0000,
        "F1 Score": 0.0000,
        "Profit Percentage": 0.00,
    },
    {
        "Strategy": "Options Straddle Strategy",
        "Accuracy": 0.5554,
        "Precision": 0.0000,
        "Recall": 0.0000,
        "F1 Score": 0.0000,
        "Profit Percentage": 0.00,
    },
    {
        "Strategy": "Gap Trading Strategy",
        "Accuracy": 0.5159,
        "Precision": 0.4557,
        "Recall": 0.4558,
        "F1 Score": 0.4557,
        "Profit Percentage": 20.27,
    },
    {
        "Strategy": "Seasonal Trading Strategy",
        "Accuracy": 0.5876,
        "Precision": 0.5346,
        "Recall": 0.5607,
        "F1 Score": 0.5474,
        "Profit Percentage": 24.93,
    },
    {
        "Strategy": "Volatility Breakout Strategy",
        "Accuracy": 0.5554,
        "Precision": 0.0000,
        "Recall": 0.0000,
        "F1 Score": 0.0000,
        "Profit Percentage": 0.00,
    },
    {
        "Strategy": "Trend Reversal Strategy",
        "Accuracy": 0.4618,
        "Precision": 0.0000,
        "Recall": 0.0000,
        "F1 Score": 0.0000,
        "Profit Percentage": 0.00,
    },
]

# Define weights for evaluation metrics
weights = {
    "Accuracy": 0.2,
    "Precision": 0.2,
    "Recall": 0.2,
    "F1 Score": 0.2,
    "Profit Percentage": 0.2
}


# Calculate overall scores for each strategy based on weighted metrics
for strategy in strategy_results:
    overall_score = (
        strategy["Accuracy"] * weights["Accuracy"] +
        strategy["Precision"] * weights["Precision"] +
        strategy["Recall"] * weights["Recall"] +
        strategy["F1 Score"] * weights["F1 Score"] +
        strategy["Profit Percentage"] * weights["Profit Percentage"]
    )
    strategy["Overall_Score"] = overall_score

# Rank strategies based on overall scores
sorted_strategies = sorted(strategy_results, key=lambda x: x["Overall_Score"], reverse=True)

# Print strategy rankings
print("Strategy Rankings:")
for rank, strategy in enumerate(sorted_strategies, start=1):
    strategy_name = strategy["Strategy"]
    overall_score = strategy["Overall_Score"]
    print(f"{rank}. {strategy_name} - Overall Score: {overall_score:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Strategy: Basic Breakout Trading
Accuracy: 0.5554
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Profit Percentage: 0.00%

Strategy: Trend Following Strategy
Accuracy: 0.4995
Precision: 0.4416
Recall: 0.4747
F1 Score: 0.4575
Profit Percentage: 21.11%

Strategy: Momentum Trading Strategy
Accuracy: 0.5606
Precision: 0.5577
Recall: 0.0564
F1 Score: 0.1024
Profit Percentage: 2.51%

Strategy: Pivot Point Trading Strategy
Accuracy: 0.8956
Precision: 0.8601
Recall: 0.9138
F1 Score: 0.8862
Profit Percentage: 40.63%



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Strategy: Range Breakout Strategy
Accuracy: 0.5554
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Profit Percentage: 0.00%



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Strategy: Options Straddle Strategy
Accuracy: 0.5554
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Profit Percentage: 0.00%

Strategy: Gap Trading Strategy
Accuracy: 0.5159
Precision: 0.4557
Recall: 0.4558
F1 Score: 0.4557
Profit Percentage: 20.27%

Strategy: Seasonal Trading Strategy
Accuracy: 0.5876
Precision: 0.5346
Recall: 0.5607
F1 Score: 0.5474
Profit Percentage: 24.93%



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Strategy: Volatility Breakout Strategy
Accuracy: 0.5554
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Profit Percentage: 0.00%

Strategy: Trend Reversal Strategy
Accuracy: 0.4618
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Profit Percentage: 0.00%

Strategy Rankings:
1. Pivot Point Trading Strategy - Overall Score: 8.8371
2. Seasonal Trading Strategy - Overall Score: 5.4321
3. Trend Following Strategy - Overall Score: 4.5967
4. Gap Trading Strategy - Overall Score: 4.4306
5. Momentum Trading Strategy - Overall Score: 0.7574
6. Basic Breakout Trading - Overall Score: 0.1111
7. Range Breakout Strategy - Overall Score: 0.1111
8. Options Straddle Strategy - Overall Score: 0.1111
9. Volatility Breakout Strategy - Overall Score: 0.1111
10. Trend Reversal Strategy - Overall Score: 0.0924
